# News Clustering

In [1]:
import pickle
import itertools
import warnings
import sys 
import os
import logging
from multiprocessing import cpu_count

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ckonlpy.tag import Twitter
from konlpy.tag import Mecab

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel, ldaseqmodel, LdaMulticore, lda_dispatcher
from gensim.models.wrappers import LdaMallet, DtmModel
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim import corpora, models, similarities
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

import pyLDAvis.gensim

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

warnings.filterwarnings('ignore')

In [2]:
def Nav_tokenizer(doc, tagger, stopwords):
    pos = tagger.pos(doc)
    pos = [word[0] for word in pos if (len(word[0])>1) & (not word[0] in stopwords)]
    return pos

In [3]:
def Nav_tokenizer_noun(doc, tagger, stopwords):
    pos = tagger.nouns(doc)
    pos = [word for word in pos if (len(word)>1) & (not word in stopwords)]
    return pos

In [4]:
def evaluate_graph(dictionary, corpus, texts, limit, tagger, modelpath):
    """
    Function to display num_topics - LDA graph using c_v coherence
    
    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : topic limit
    
    Returns:
    -------
    lm_list : List of LDA topic models
    c_v : Coherence values corresponding to the LDA model with respective number of topics
    """
    c_v = []
    lm_list = []
    import os
    for num_topics in range(1, limit):
        ldaFileName = os.path.join(modelpath, 'ldamodel_{}_topic-{}'.format(tagger, num_topics))
        cohFileName = os.path.join(modelpath, 'coherencemodel_{}_topic-{}'.format(tagger, num_topics))
        if not os.path.isfile(ldaFileName):
            lm = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
            lm.save(ldaFileName)
        else:
            lm = LdaModel.load(ldaFileName)
        lm_list.append(lm)
        if not os.path.isfile(cohFileName):
            cm = CoherenceModel(model=lm, texts=texts, dictionary=dictionary, coherence='c_v')
            cm.save(cohFileName)
        else:
            cm = CoherenceModel.load(cohFileName)
        c_v.append(cm.get_coherence())
        
    # Show graph
    x = range(1, limit)
    plt.plot(x, c_v)
    plt.xlabel("num_topics")
    plt.ylabel("Coherence score")
    plt.legend(("c_v"), loc='best')
    plt.show()
    
    return lm_list, c_v

### Stopwords

In [5]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## News

### Naver

In [6]:
dictNaver = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
dfNaver = pd.DataFrame.from_dict(dictNaver, orient='index')
print (dfNaver.shape)

(15120, 10)


### Daum

In [7]:
dictDaum = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
dfDaum = pd.DataFrame.from_dict(dictDaum, orient='index')
print (dfDaum.shape)

(9372, 10)


## Daum

### 뉴스 기사 통합

In [8]:
combinedDf = pd.concat([dfNaver, dfDaum])
combinedDf.head()

,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
5a29c445588c132954d1973a,정치,2017.12.07,연합뉴스,1713,1465,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","[외무성, 핵전쟁, 대변인]","{조선반도, 중앙, 북한, 도화선, 고위, 대변인, 미국, 핵전쟁}"
5a29c445588c132954d1973b,정치,2017.12.07,한국일보,2551,2062,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"[예산, 예결위, soc]","{증액, 국민의당, 예산안, 지역구, 의원, 정부안, 호남}"
5a29c445588c132954d1973c,정치,2017.12.07,뉴시스,610,536,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"[최경환, 구속영장, 국가정보원]","{국정원장, 조사, 혐의, 의원, 정기국회, 검찰, 구속영장 청구}"
5a29c445588c132954d1973d,정치,2017.12.07,연합뉴스,145,133,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"[이슈 · 최재형 감사원장 내정, 감사원장, 최재형, 감사원]","{법관, 지명, 후보자, 공직 사회, 생활, 감사원장}"
5a29c445588c132954d1973e,정치,2017.12.07,동아일보,1074,932,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"[김정은, b-1b, 한반도]","{사무차장, 양강도, 시찰, 공장, 접경, 펠트먼, 훈련, 김정은, 삼지연}"


In [9]:
extKeywords = combinedDf.extracted_keywords.tolist()

In [10]:
if sys.platform =='darwin':
    clusteringPath ='/Volumes/disk1/Clustering/'
    clusteringModelPath = '/Volumes/disk1/Clustering_model/'
elif sys.platform =='win32':
    clusteringPath = 'd:/Clustering/' 
    clusteringModelPath = 'd:/Clustering_model/'

### 사전 데이터 제작

In [11]:
%%time
dict_keywords_name = clusteringModelPath + 'dictionary_keywords'
if not os.path.isfile(dict_keywords_name):
    dict_keywords = Dictionary(extKeywords)
    dict_keywords.save(dict_keywords_name)
else:
    dict_keywords = Dictionary.load(dict_keywords_name)

CPU times: user 23.1 ms, sys: 5.5 ms, total: 28.6 ms
Wall time: 412 ms


In [12]:
%%time
corpus_keywords_name = clusteringModelPath + 'corpus_keywords.pickled'
if not os.path.isfile(corpus_keywords_name):
    corpus_keywords = [ dict_keywords.doc2bow(text) for text in tqdm(extKeywords)]
    pickle.dump(corpus_keywords, open(corpus_keywords_name, 'wb'))
else:
    corpus_keywords = pickle.load(open(corpus_keywords_name, 'rb'))

CPU times: user 176 ms, sys: 12.7 ms, total: 189 ms
Wall time: 213 ms


In [13]:
print('Number of unique tokens: %d' % len(dict_keywords))
print('Number of documents: %d' % len(corpus_keywords))

Number of unique tokens: 32800
Number of documents: 24492


### LSI (  Latent Semantic Indexing )
* an indexing and retrieval method that uses a mathematical technique called singular value decomposition (SVD) to identify patterns in the relationships between the terms and concepts contained in an unstructured collection of text  

In [14]:
%%time
lsimodel_keywords_name = clusteringModelPath + 'lsimodel_keywords'
if not os.path.isfile(lsimodel_keywords_name):
    lsimodel_keywords = LsiModel(corpus = corpus_keywords, num_topics = 20, id2word = dict_keywords)
    lsimodel_keywords.save(lsimodel_keywords_name)
else:
    lsimodel_keywords = LsiModel.load(lsimodel_keywords_name)

CPU times: user 55.1 ms, sys: 18.4 ms, total: 73.5 ms
Wall time: 156 ms


In [15]:
lsimodel_keywords.show_topics(num_topics = 20)

[(0,
  '0.520*"선수" + 0.516*"경기" + 0.329*"한국" + 0.174*"미국" + 0.151*"계약" + 0.142*"감독" + 0.127*"중국" + 0.124*"일본" + 0.122*"기록" + 0.111*"손흥민"'),
 (1,
  '0.400*"대통령" + 0.327*"미국" + 0.319*"정부" + -0.298*"선수" + 0.256*"한국" + 0.253*"서울" + 0.225*"중국" + -0.223*"경기" + 0.198*"북한" + 0.193*"검찰"'),
 (2,
  '-0.690*"서울" + 0.295*"한국" + 0.281*"미국" + -0.257*"기온" + -0.177*"한파" + -0.162*"날씨" + 0.154*"중국" + -0.142*"추위" + -0.133*"지역" + 0.127*"북한"'),
 (3,
  '0.455*"선수" + -0.417*"경기" + -0.330*"한국" + 0.283*"대통령" + 0.248*"계약" + 0.212*"검찰" + 0.176*"구단" + 0.170*"FA" + -0.148*"서울" + 0.134*"혐의"'),
 (4,
  '0.416*"경기" + 0.364*"대통령" + -0.355*"미국" + 0.308*"검찰" + -0.233*"서울" + -0.198*"선수" + 0.182*"혐의" + 0.151*"수사" + -0.150*"계약" + 0.148*"청와대"'),
 (5,
  '-0.694*"방송" + -0.450*"모습" + -0.209*"한국" + 0.185*"미국" + 0.167*"경기" + -0.157*"사진" + 0.109*"정부" + -0.108*"공개" + -0.107*"드라마" + -0.105*"경찰"'),
 (6,
  '-0.613*"한국" + 0.533*"미국" + 0.259*"방송" + 0.238*"경기" + 0.201*"북한" + -0.161*"일본" + 0.159*"모습" + -0.085*"정부" + 0.085*"토트넘" + -0.084*"선

In [16]:
lsitopics_keywords = lsimodel_keywords.show_topics(formatted = False)

### HDP (Hierarchical Dirichlet Process)
* a non-parametric bayesian method (note the missing number of requested topics)

In [17]:
%%time
hdpmodel_keywords_name = clusteringModelPath+'hdpmodel_keywords'
if not os.path.isfile(hdpmodel_keywords_name):
    hdpmodel_keywords = HdpModel(corpus = corpus_keywords, id2word = dict_keywords)
    hdpmodel_keywords.save(clusteringModelPath+'hdpmodel_keywords')
else:
    hdpmodel_keywords = HdpModel.load(hdpmodel_keywords_name)

CPU times: user 845 ms, sys: 266 ms, total: 1.11 s
Wall time: 2.75 s


In [18]:
hdpmodel_keywords.show_topics()

[(0,
  '0.004*선수 + 0.002*계약 + 0.002*경기 + 0.002*FA + 0.001*롯데 + 0.001*한국 + 0.001*구단 + 0.001*투수 + 0.001*정부 + 0.001*KIA + 0.001*두산 + 0.001*일본 + 0.001*감독 + 0.001*연봉 + 0.001*북한 + 0.001*LG + 0.001*영입 + 0.001*리그 + 0.001*미국 + 0.001*대통령'),
 (1,
  '0.003*경기 + 0.002*한국 + 0.002*선수 + 0.001*축구 + 0.001*대회 + 0.001*감독 + 0.001*토트넘 + 0.001*기록 + 0.001*산체스 + 0.001*서울 + 0.001*맨유 + 0.001*월드컵 + 0.001*손흥민 + 0.001*일본 + 0.001*중국 + 0.001*계약 + 0.001*이적 + 0.001*리그 + 0.001*관심 + 0.001*대표팀'),
 (2,
  '0.001*경기 + 0.001*선수 + 0.001*서울 + 0.001*대통령 + 0.001*한국 + 0.001*정부 + 0.001*미국 + 0.001*대표 + 0.001*의원 + 0.001*검찰 + 0.001*사고 + 0.001*방송 + 0.001*중국 + 0.001*일본 + 0.001*청와대 + 0.001*기록 + 0.001*경찰 + 0.001*의혹 + 0.001*모습 + 0.000*혐의'),
 (3,
  '0.004*서울 + 0.003*기온 + 0.002*전국 + 0.002*날씨 + 0.002*한파 + 0.002*추위 + 0.001*영하 + 0.001*지역 + 0.001*아침 + 0.001*경기 + 0.001*선수 + 0.001*지방 + 0.001*중국 + 0.001*바람 + 0.001*대통령 + 0.001*남부 + 0.001*미세먼지 + 0.001*모습 + 0.001*공기 + 0.001*경찰'),
 (4,
  '0.002*선수 + 0.001*경기 + 0.001*서울 + 0.001*대통령 + 0.001*혐의 + 0.001*북한

In [19]:
hdptopics_keywords = hdpmodel_keywords.show_topics(formatted = False)

### LDA (Latent Dirichlet allocation)
* a generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar  

#### LDA model1
* basic

In [20]:
pyLDAvis.enable_notebook()

In [21]:
pl_keywords = PerplexityMetric(corpus = corpus_keywords, logger = 'shell', 
                        title = 'Perplexity (twitter)')
ch_umass_keywords = CoherenceMetric(corpus = corpus_keywords, coherence = 'u_mass', 
                             logger = 'shell', title = ' Coherence (u_mass)')
ch_cv_keywords = CoherenceMetric(corpus = corpus_keywords, logger = 'shell', 
                          texts = extKeywords, coherence = 'c_v', 
                          title = 'Coherence (c_v)')
diff_kl_keywords = DiffMetric(distance = 'kullback_leibler', 
                       logger = 'shell', title = 'Diff (kullback_leibler)')
convergence_kl_keywords = ConvergenceMetric(distance = 'jaccard', logger = 'shell', 
                                     title = 'Convergence (jaccard)')
callbacks_keywords = [pl_keywords, ch_umass_keywords, ch_cv_keywords, diff_kl_keywords, convergence_kl_keywords]

In [22]:
%%time
logging.basicConfig(level = logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
ldamodel_keywords_name = clusteringModelPath + 'ldamodel_keywords'
if not os.path.isfile(ldamodel_keywords_name):
    ldamodel_keywords = LdaModel(corpus = corpus_keywords, num_topics = 20,
                           id2word = dict_keywords, passes = 50,
                           chunksize = 6123, iterations = 250,
                           alpha='auto', callbacks = callbacks_keywords)
    ldamodel_keywords.save(ldamodel_keywords_name)
else:
    ldamodel_keywords = LdaModel.load(ldamodel_keywords_name)

INFO:gensim.utils:loading LdaModel object from /Volumes/disk1/Clustering_model/ldamodel_keywords
INFO:gensim.utils:loading expElogbeta from /Volumes/disk1/Clustering_model/ldamodel_keywords.expElogbeta.npy with mmap=None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:setting ignored attribute id2word to None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:loaded /Volumes/disk1/Clustering_model/ldamodel_keywords
INFO:gensim.utils:loading LdaState object from /Volumes/disk1/Clustering_model/ldamodel_keywords.state
INFO:gensim.utils:loaded /Volumes/disk1/Clustering_model/ldamodel_keywords.state


CPU times: user 12.7 s, sys: 1.46 s, total: 14.2 s
Wall time: 16.6 s


In [23]:
%%time
coherence1_um_keywords_name = clusteringModelPath + 'coherence1_keywords_u_mass'
if not os.path.isfile(coherence1_um_keywords_name):
    cm_keywords = CoherenceModel(model = ldamodel_keywords, 
                      corpus = corpus_keywords, 
                      dictionary = dict_keywords,
                      coherence = 'u_mass')
    cm_keywords.save(coherence1_um_keywords_name)
else:
    cm_keywords = CoherenceModel.load(coherence1_um_keywords_name)

INFO:gensim.utils:loading CoherenceModel object from /Volumes/disk1/Clustering_model/coherence1_keywords_u_mass
INFO:gensim.utils:loading dictionary recursively from /Volumes/disk1/Clustering_model/coherence1_keywords_u_mass.dictionary.* with mmap=None
INFO:gensim.utils:loading _model recursively from /Volumes/disk1/Clustering_model/coherence1_keywords_u_mass._model.* with mmap=None
INFO:gensim.utils:loading id2word recursively from /Volumes/disk1/Clustering_model/coherence1_keywords_u_mass._model.id2word.* with mmap=None
INFO:gensim.utils:loading state recursively from /Volumes/disk1/Clustering_model/coherence1_keywords_u_mass._model.state.* with mmap=None
INFO:gensim.utils:loaded /Volumes/disk1/Clustering_model/coherence1_keywords_u_mass


CPU times: user 15.1 s, sys: 3.29 s, total: 18.4 s
Wall time: 21.3 s


In [24]:
print ('Coherence : {}'.format(cm_keywords.get_coherence()))

INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 7000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 8000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 9000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 10000 documents
INFO:gensim.topic_c

Coherence : -10.899798791331238


In [25]:
%%time
coherence1_cv_keywords_name = clusteringModelPath + 'coherence1_keywords_c_v'
if not os.path.isfile(coherence1_cv_keywords_name):
    cm_keywords_cv = CoherenceModel(model = ldamodel_keywords, 
                         texts = extKeywords,
                         dictionary = dict_keywords, 
                         coherence = 'c_v')
    cm_keywords_cv.save(coherence1_cv_keywords_name)
else:
    cm_keywords_cv = CoherenceModel.load(coherence1_cv_keywords_name)

INFO:gensim.utils:loading CoherenceModel object from /Volumes/disk1/Clustering_model/coherence1_keywords_c_v
INFO:gensim.utils:loading dictionary recursively from /Volumes/disk1/Clustering_model/coherence1_keywords_c_v.dictionary.* with mmap=None
INFO:gensim.utils:loading _model recursively from /Volumes/disk1/Clustering_model/coherence1_keywords_c_v._model.* with mmap=None
INFO:gensim.utils:loading id2word recursively from /Volumes/disk1/Clustering_model/coherence1_keywords_c_v._model.id2word.* with mmap=None
INFO:gensim.utils:loading state recursively from /Volumes/disk1/Clustering_model/coherence1_keywords_c_v._model.state.* with mmap=None
INFO:gensim.utils:loaded /Volumes/disk1/Clustering_model/coherence1_keywords_c_v


CPU times: user 13.7 s, sys: 5.2 s, total: 18.9 s
Wall time: 22.1 s


In [26]:
print ('Coherence : {}'.format(cm_keywords_cv.get_coherence()))

INFO:gensim.topic_coherence.probability_estimation:using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.text_analysis:serializing accumulator to return to master...
INFO:gensim.topic_coherence.text_analysis:serializing accumulator to return to master...
INFO:gensim.topic_coherence.text_analysis:serializing accumulator to return to master...
INFO:gensim.topic_coherence.text_analysis:accumulator serialized
INFO:gensim.topic_coherence.text_analysis:accumulator serialized
INFO:gensim.topic_coherence.text_analysis:accumulator serialized
INFO:gensim.topic_coherence.text_analysis:3 accumulators retrieved from output queue
INFO:gensim.topic_coherence.text_analysis:accumulated word occurrence stats for 21234 virtual documents


Coherence : 0.4587076374923855


In [27]:
%%time
pyLDAvis.gensim.prepare(ldamodel_keywords, corpus_keywords, dict_keywords)

CPU times: user 24.1 s, sys: 7.04 s, total: 31.2 s
Wall time: 40.3 s


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
16     7.239108        1       1 -0.090976 -0.199698
10     6.188592        1       2  0.057597 -0.150182
12     6.094420        1       3  0.105869  0.182852
5      6.066524        1       4 -0.170829 -0.047774
2      5.915551        1       5 -0.230248  0.114455
3      5.823819        1       6 -0.235702  0.071461
6      5.665359        1       7 -0.044927 -0.194853
14     5.571172        1       8  0.054922 -0.091159
13     5.211153        1       9  0.088789  0.018917
19     5.206195        1      10  0.119551  0.106569
1      5.113411        1      11 -0.191677  0.127845
8      4.472063        1      12  0.038091 -0.100263
0      4.463650        1      13  0.028861 -0.013446
18     4.400042        1      14  0.080587  0.053011
7      3.981029        1      15  0.005508  0.023869
4      3.900302        1      16  0.064938  0.021697
11     3.895743        1      17  0.111323  0.039872
15     3.734555        1      18  0.093787  0.107937
17     3.531055        1      19  0.052012 -0.073812
9      3.526255        1      20  0.062525  0.002702, topic_info=      Category         Freq   Term        Total  loglift  logprob
term                                                             
41     Default   867.000000     서울   867.000000  30.0000  30.0000
169    Default  1087.000000     경기  1087.000000  29.0000  29.0000
227    Default   971.000000     한국   971.000000  28.0000  28.0000
1116   Default  1094.000000     선수  1094.000000  27.0000  27.0000
15     Default   506.000000     검찰   506.000000  26.0000  26.0000
1126   Default   502.000000     계약   502.000000  25.0000  25.0000
3      Default   863.000000     미국   863.000000  24.0000  24.0000
36     Default   760.000000    대통령   760.000000  23.0000  23.0000
92     Default   405.000000     대표   405.000000  22.0000  22.0000
87     Default   762.000000     방송   762.000000  21.0000  21.0000
50     Default   800.000000     정부   800.000000  20.0000  20.0000
590    Default   537.000000     일본   537.000000  19.0000  19.0000
10     Default   370.000000     의원   370.000000  18.0000  18.0000
1113   Default   330.000000     구단   330.000000  17.0000  17.0000
51     Default   330.000000    청와대   330.000000  16.0000  16.0000
1783   Default   310.000000     화재   310.000000  15.0000  15.0000
1148   Default   430.000000     기록   430.000000  14.0000  14.0000
4      Default   570.000000     북한   570.000000  13.0000  13.0000
1137   Default   298.000000    토트넘   298.000000  12.0000  12.0000
20     Default   426.000000     혐의   426.000000  11.0000  11.0000
1051   Default   294.000000    드라마   294.000000  10.0000  10.0000
63     Default   333.000000     지역   333.000000   9.0000   9.0000
1132   Default   288.000000    손흥민   288.000000   8.0000   8.0000
1151   Default   287.000000     FA   287.000000   7.0000   7.0000
593    Default   275.000000     기온   275.000000   6.0000   6.0000
553    Default   259.000000     한파   259.000000   5.0000   5.0000
327    Default   578.000000     경찰   578.000000   4.0000   4.0000
1059   Default   289.000000     배우   289.000000   3.0000   3.0000
175    Default   269.000000  가상 화폐   269.000000   2.0000   2.0000
338    Default   316.000000     사고   316.000000   1.0000   1.0000
...        ...          ...    ...          ...      ...      ...
8001   Topic20    15.831225   가상계좌    16.634274   3.2955  -6.0065
3144   Topic20    15.389618     총재    16.192665   3.2941  -6.0348
1028   Topic20    15.374360     추억    16.177408   3.2940  -6.0358
10907  Topic20    15.278608   드래프트    16.081656   3.2937  -6.0420
3141   Topic20    14.875161  금리 인상    15.678208   3.2924  -6.0688
8961   Topic20    14.565915     배달    15.368962   3.2913  -6.0898
25949  Topic20    14.250746    진해림    15.053793   3.2901  -6.1117
1298   Topic20    14.235146     심사    15.038193   3.2901  -6.1128
3370   Topic20    14.177895    KTX    14.980942   3.2898  -6.1168
3669   Topic20    1

In [28]:
ldatopics_keywords = ldamodel_keywords.show_topics(formatted = False)

##### display num_topics - LDA graph using c_v coherence

In [ ]:
%%time
lmlist_keywords, c_v_keywords = evaluate_graph(dictionary = dict_keywords, corpus = corpus_keywords, texts = extKeywords, limit = 100, tagger = 'mixed', modelpath=clusteringModelPath)

INFO:gensim.utils:loading LdaModel object from /Volumes/disk1/Clustering_model/ldamodel_mixed_topic-1
INFO:gensim.utils:loading expElogbeta from /Volumes/disk1/Clustering_model/ldamodel_mixed_topic-1.expElogbeta.npy with mmap=None
INFO:gensim.utils:setting ignored attribute id2word to None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:loaded /Volumes/disk1/Clustering_model/ldamodel_mixed_topic-1
INFO:gensim.utils:loading LdaState object from /Volumes/disk1/Clustering_model/ldamodel_mixed_topic-1.state
INFO:gensim.utils:loaded /Volumes/disk1/Clustering_model/ldamodel_mixed_topic-1.state
INFO:gensim.utils:loading CoherenceModel object from /Volumes/disk1/Clustering_model/coherencemodel_mixed_topic-1
INFO:gensim.utils:loading dictionary recursively from /Volumes/disk1/Clustering_model/coherencemodel_mixed_topic-1.dictionary.* with mmap=None
INFO:gensim.utils:loading _model recursively from /Volume

INFO:gensim.topic_coherence.probability_estimation:using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sliding windows
INFO:gensim.topic_coherence.text_analysis:serializing accumulator to return to master...
INFO:gensim.topic_coherence.text_analysis:serializing accumulator to return to master...
INFO:gensim.topic_coherence.text_analysis:serializing accumulator to return to master...
INFO:gensim.topic_coherence.text_analysis:accumulator serialized
INFO:gensim.topic_coherence.text_analysis:accumulator serialized
INFO:gensim.topic_coherence.text_analysis:accumulator serialized
INFO:gensim.topic_coherence.text_analysis:3 accumulators retrieved from output queue
INFO:gensim.topic_coherence.text_analysis:accumulated word occurrence stats for 15319 virtual documents
INFO:gensim.utils:loading LdaModel object from /Volumes/disk1/Clustering_model/ldamodel_mixed_topic-5
INFO:gensim.utils:loading expElogbeta from /Volumes/disk1/Clustering_model/ldamod

INFO:gensim.utils:loaded /Volumes/disk1/Clustering_model/ldamodel_mixed_topic-8.state
INFO:gensim.utils:loading CoherenceModel object from /Volumes/disk1/Clustering_model/coherencemodel_mixed_topic-8
INFO:gensim.utils:loading dictionary recursively from /Volumes/disk1/Clustering_model/coherencemodel_mixed_topic-8.dictionary.* with mmap=None
INFO:gensim.utils:loading _model recursively from /Volumes/disk1/Clustering_model/coherencemodel_mixed_topic-8._model.* with mmap=None
INFO:gensim.utils:loading id2word recursively from /Volumes/disk1/Clustering_model/coherencemodel_mixed_topic-8._model.id2word.* with mmap=None
INFO:gensim.utils:loading state recursively from /Volumes/disk1/Clustering_model/coherencemodel_mixed_topic-8._model.state.* with mmap=None
INFO:gensim.utils:loaded /Volumes/disk1/Clustering_model/coherencemodel_mixed_topic-8
INFO:gensim.topic_coherence.probability_estimation:using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sl

INFO:gensim.models.ldamodel:topic #1 (0.100): 0.028*"서울" + 0.015*"기온" + 0.014*"전국" + 0.011*"추위" + 0.010*"날씨" + 0.009*"한파" + 0.009*"아침" + 0.008*"지역" + 0.008*"거래소" + 0.007*"영하"
INFO:gensim.models.ldamodel:topic #3 (0.100): 0.012*"경기" + 0.007*"서비스" + 0.005*"선수" + 0.005*"기술" + 0.005*"감독" + 0.005*"사용" + 0.005*"2017" + 0.004*"대출" + 0.004*"학교" + 0.004*"개봉"
INFO:gensim.models.ldamodel:topic #7 (0.100): 0.023*"북한" + 0.012*"미국" + 0.009*"손흥민" + 0.008*"영화" + 0.008*"배우" + 0.007*"중국" + 0.007*"드라마" + 0.007*"제재" + 0.007*"토트넘" + 0.007*"화재"
INFO:gensim.models.ldamodel:topic diff=0.347252, rho=0.500000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #10000/24492
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 24492 documents
INFO:gensim.models.ldamodel:topic #2 (0.100): 0.014*"경찰" + 0.013*"정부" + 0.010*"혐의" + 0.010*"화재" + 0.007*"아파트" + 0.007*"대통령" + 0.006*"청와대" + 0.005*"중국" + 0.005*"건물" + 0.005*"사건"
INFO:gensim.models.ldamodel:topic #8 (0.100): 0.020*"선수" + 0.014*

INFO:gensim.models.ldamodel:topic #2 (0.100): 0.015*"경찰" + 0.013*"화재" + 0.010*"혐의" + 0.009*"정부" + 0.007*"사건" + 0.007*"아파트" + 0.006*"건물" + 0.006*"평균자책점" + 0.006*"멤버" + 0.006*"소속사"
INFO:gensim.models.ldamodel:topic diff=0.332403, rho=0.301511
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #24000/24492
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 24492 documents
INFO:gensim.models.ldamodel:topic #6 (0.100): 0.037*"방송" + 0.023*"모습" + 0.010*"공개" + 0.010*"사랑" + 0.010*"사진" + 0.009*"예능" + 0.009*"프로그램" + 0.007*"생활" + 0.007*"이야기" + 0.007*"엄마"
INFO:gensim.models.ldamodel:topic #4 (0.100): 0.012*"한국" + 0.009*"평창" + 0.008*"중국" + 0.008*"올림픽" + 0.008*"시장" + 0.007*"미국" + 0.007*"세트" + 0.006*"방송" + 0.006*"국내" + 0.006*"호주"
INFO:gensim.models.ldamodel:topic #0 (0.100): 0.015*"검찰" + 0.013*"경기" + 0.012*"대회" + 0.012*"수사" + 0.010*"상황" + 0.009*"이적" + 0.008*"아내" + 0.008*"23" + 0.008*"tvN" + 0.007*"맨유"
INFO:gensim.models.ldamodel:topic #2 (0.100): 0.015*"화재" + 0.015*

...

INFO:gensim.models.ldamodel:topic diff=0.362377, rho=0.301511
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #24000/24492
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 24492 documents
INFO:gensim.models.ldamodel:topic #4 (0.038): 0.047*"북한" + 0.014*"남북" + 0.014*"대통령" + 0.013*"중국" + 0.013*"미국" + 0.012*"장면" + 0.012*"행사" + 0.011*"평창올림픽" + 0.010*"정부" + 0.010*"사진"
INFO:gensim.models.ldamodel:topic #17 (0.038): 0.062*"화재" + 0.028*"통합" + 0.015*"공식" + 0.014*"사업" + 0.013*"투어" + 0.011*"제천" + 0.009*"뉴욕" + 0.008*"미국" + 0.008*"전쟁" + 0.008*"참사"
INFO:gensim.models.ldamodel:topic #3 (0.038): 0.037*"일본" + 0.025*"서울" + 0.019*"지역" + 0.017*"한국" + 0.014*"SNS" + 0.014*"1987" + 0.012*"손님" + 0.012*"전국" + 0.011*"영화" + 0.011*"UAE"
INFO:gensim.models.ldamodel:topic #15 (0.038): 0.024*"활약" + 0.020*"영화" + 0.018*"민주당" + 0.017*"도움" + 0.012*"후보" + 0.012*"사망" + 0.010*"성폭행" + 0.010*"대학" + 0.009*"유로" + 0.009*"기간"
INFO:gensim.models.ldamodel:topic #5 (0.038): 0.040*"대회" + 0.01

INFO:gensim.models.ldamodel:topic #24 (0.037): 0.022*"통합" + 0.017*"미국" + 0.015*"정부" + 0.010*"대통령" + 0.009*"열차" + 0.009*"서비스" + 0.008*"검토" + 0.008*"제도" + 0.007*"중단" + 0.007*"외교"
INFO:gensim.models.ldamodel:topic #11 (0.037): 0.027*"아파트" + 0.021*"크리스마스" + 0.015*"경찰" + 0.015*"병원" + 0.015*"건물" + 0.012*"화유기" + 0.011*"서울" + 0.011*"사고" + 0.011*"도로" + 0.011*"양현종"
INFO:gensim.models.ldamodel:topic #3 (0.037): 0.047*"경기" + 0.032*"기록" + 0.029*"선수" + 0.023*"비트코인" + 0.021*"가상 화폐" + 0.016*"가격" + 0.015*"감독" + 0.014*"맨시티" + 0.013*"한국" + 0.013*"LG"
INFO:gensim.models.ldamodel:topic #1 (0.037): 0.035*"검찰" + 0.031*"수사" + 0.027*"대통령" + 0.024*"혐의" + 0.021*"북한" + 0.019*"청와대" + 0.016*"의혹" + 0.013*"광주" + 0.010*"서울" + 0.010*"김정은"
INFO:gensim.models.ldamodel:topic diff=0.366924, rho=0.500000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #10000/24492
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 24492 documents
INFO:gensim.models.ldamodel:topic #7 (0.037): 0.017*"경찰"

INFO:gensim.models.ldamodel:topic #11 (0.037): 0.028*"경찰" + 0.026*"사고" + 0.023*"병원" + 0.023*"아파트" + 0.019*"작품" + 0.018*"20" + 0.016*"넥센" + 0.015*"세상" + 0.015*"건물" + 0.014*"화유기"
INFO:gensim.models.ldamodel:topic #1 (0.037): 0.065*"검찰" + 0.046*"대통령" + 0.041*"수사" + 0.039*"혐의" + 0.031*"청와대" + 0.022*"사건" + 0.021*"경찰" + 0.018*"국정원" + 0.017*"의혹" + 0.011*"의원"
INFO:gensim.models.ldamodel:topic diff=0.361531, rho=0.301511
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #24000/24492
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 24492 documents
INFO:gensim.models.ldamodel:topic #6 (0.037): 0.017*"홈런" + 0.017*"성적" + 0.016*"호주" + 0.013*"음식" + 0.012*"황금빛" + 0.012*"타율" + 0.011*"발언" + 0.011*"타자" + 0.011*"건강" + 0.009*"모습"
INFO:gensim.models.ldamodel:topic #4 (0.037): 0.029*"방송" + 0.022*"베트남" + 0.017*"촬영" + 0.017*"무한도전" + 0.015*"출연" + 0.014*"만원" + 0.014*"예능" + 0.014*"언론" + 0.014*"모습" + 0.012*"프로그램"
INFO:gensim.models.ldamodel:topic #5 (0.037): 0.043*"토트넘" + 0.0

INFO:gensim.models.ldamodel:topic diff=0.417373, rho=0.577350
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #8000/24492
INFO:gensim.models.ldamodel:merging changes from 2000 documents into a model of 24492 documents
INFO:gensim.models.ldamodel:topic #10 (0.036): 0.011*"승객" + 0.010*"미국" + 0.009*"통신" + 0.009*"관광객" + 0.008*"부부" + 0.008*"KT" + 0.008*"부모" + 0.007*"수입" + 0.007*"대통령" + 0.007*"정규직 전환"
INFO:gensim.models.ldamodel:topic #15 (0.036): 0.017*"한국당" + 0.014*"눈물" + 0.011*"직원" + 0.011*"결혼" + 0.010*"고객" + 0.008*"세대" + 0.008*"무한도전" + 0.007*"진심" + 0.007*"의원" + 0.007*"멤버들"
INFO:gensim.models.ldamodel:topic #2 (0.036): 0.017*"영화" + 0.015*"고준희" + 0.012*"원전" + 0.011*"계약" + 0.011*"사진" + 0.010*"이름" + 0.010*"지원" + 0.010*"KIA" + 0.009*"개봉" + 0.008*"배우"
INFO:gensim.models.ldamodel:topic #19 (0.036): 0.017*"30" + 0.014*"광주" + 0.014*"세계" + 0.013*"18" + 0.013*"서비스" + 0.011*"주변" + 0.011*"개통" + 0.010*"의견" + 0.010*"기술" + 0.010*"레알"
INFO:gensim.models.ldamodel:topic #5 (0.036): 0.027*"투표" + 0

INFO:gensim.models.ldamodel:topic #11 (0.036): 0.021*"인상" + 0.019*"최저 임금" + 0.018*"기온" + 0.015*"날씨" + 0.013*"서울" + 0.013*"아침" + 0.013*"추위" + 0.012*"겨울" + 0.012*"대구" + 0.011*"최고"
INFO:gensim.models.ldamodel:topic #24 (0.036): 0.036*"경기" + 0.028*"맨유" + 0.019*"맨시티" + 0.019*"부상" + 0.016*"kt" + 0.014*"아스널" + 0.013*"남성" + 0.013*"UAE" + 0.011*"감독" + 0.010*"여성"
INFO:gensim.models.ldamodel:topic #15 (0.036): 0.052*"결혼" + 0.023*"눈물" + 0.021*"한국당" + 0.015*"직원" + 0.014*"방송" + 0.013*"멤버들" + 0.010*"출연" + 0.009*"무한도전" + 0.008*"사랑" + 0.008*"고객"
INFO:gensim.models.ldamodel:topic #25 (0.036): 0.053*"북한" + 0.017*"남북" + 0.017*"미국" + 0.015*"니퍼트" + 0.015*"중국" + 0.012*"신고" + 0.012*"법원" + 0.012*"얼굴" + 0.011*"행사" + 0.010*"나이"
INFO:gensim.models.ldamodel:topic #19 (0.036): 0.021*"레알" + 0.019*"30" + 0.018*"11" + 0.016*"손님" + 0.015*"광주" + 0.013*"과거" + 0.013*"평가" + 0.012*"기술" + 0.012*"24" + 0.011*"호날두"
INFO:gensim.models.ldamodel:topic diff=0.359270, rho=0.301511
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at do